In [12]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import os

In [2]:
#initializing networks
#All code taken from https://www.youtube.com/watch?v=IT53xBR1A7M tutorial

mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40)
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40)
resnet = InceptionResnetV1(pretrained='vggface2').eval()

100.0%


In [21]:
dataset = datasets.ImageFolder('photos') #photos folder path -> todo create photos folder and add facial recognition
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} #accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = []
name_list = []
embedding_list = []

for img, idx in loader:
    face, prob = mtcnn0(img, return_prob=True)
    if face is not None and prob>0.92:
        emb = resnet(face.unsqueeze(0))
        embedding_list.append(emb.detach())
        name_list.append(idx_to_class[idx])

#savedata
data = [embedding_list, name_list]
torch.save(data, 'data.pt')

In [23]:
#using webcam to recognize face

#loading data.pt file
load_data = torch.load('data.pt')
embedding_list = load_data[0]
name_list = load_data[1]

cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    if not ret:
        print("fail to grab frame, try again")
        break

    img = Image.fromarray(frame)
    img_cropped_list, prob_list = mtcnn(img, return_prob=True)

    if img_cropped_list is not None:
        boxes, _ = mtcnn.detect(img)

        for i, prob in enumerate(prob_list):
            if prob>0.90:
                emb - resnet(img_cropped_list[i].unsqueeze(0)).detach()

                dist_list = []

                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list)
                min_dist_idx = dist_list.index(min_dist)
                name = name_list[min_dist_idx]

                box = boxes[i]

                original_frame = frame.copy()

                if min_dist<0.9:
                    frame = cv2.putText(frame, name+' '+str(min_dist), (0,185), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA)
                    #                    frame = cv2.putText(frame, name+' '+str(min_dist), (box[0],box[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA)
                

                frame =  cv2.rectangle(frame, (0,0), (240,240), (255,0,0), 2)

    cv2.imshow("IMG", frame)

    k = cv2.waitKey(1)
    if k%256==27:
        print("Esc pressed, closing...")
        break
    elif k%256==32:
        print('Enter your name :')
        name = input()

        if not os.path.exists('photos/'+name):
            os.mkdir('photos/'+name)

        img_name = "photos/{}/{}.jpg".format(name, int(time.tie()))
        cv2.imgwrite(img_name, original_frame)
        print(" saved: {}".format(img_name))

cam.release()
cv2.destroyAllWindows()



Esc pressed, closing...


In [11]:
cam.release()
cv2.destroyAllWindows()